# Atmospheric reaction model used by Sandu (2001)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg

from numba import jit, float64, stencil


rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()

ck5 = rk.loadRKM('CK5').__num__()
dp5 = rk.loadRKM('DP5').__num__()
pd8 = rk.loadRKM('PD8').__num__()


ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()


from OrderCondition import *
from RKimple import *
import utils

# Define TR-BDF2

Define the the 'TR-BDF2'-Method from Bank et al. (In formulation of Hosea and Shampine) 
http://www.sciencedirect.com/science/article/pii/0168927495001158

To get an Error estiamtor we define it as a RK-Pair

In [ ]:
import nodepy.snp as snp

class ImplicitRungeKuttaPair(rk.RungeKuttaMethod):
    def __init__(self,A=None,b=None,bhat=None,
            name='Runge-Kutta Pair',shortname='RKM',description='',order=(None,None)):
        r"""
            In addition to the ordinary Runge-Kutta initialization,
            here the embedded coefficients `\hat{b}_j` are set as well.
        """
        super(ImplicitRungeKuttaPair,self).__init__(
                        A,b,shortname=shortname,description=description,order=order[0])
        if bhat.shape != self.b.shape:
            raise Exception("Dimensions of embedded method don't agree with those of principal method")
        self.bhat     = bhat
        self.mtype = 'Implicit embedded Runge-Kutta pair'
        self._p_hat = order[1]    

    def __num__(self):
        """
        Returns a copy of the method but with floating-point coefficients.
        This is useful whenever we need to operate numerically without
        worrying about the representation of the method.
        """
        numself = super(ImplicitRungeKuttaPair,self).__num__()
        if self.A.dtype==object:
            numself.bhat=np.array(self.bhat,dtype=np.float64)
        return numself

def load_orig():
    from sympy import sqrt, Rational

    RK={}

    half = Rational(1,2)
    one  = Rational(1,1)
    zero = Rational(0,1)
    
    A=snp.array([[0,0,0],[one/4,one/4,0],[one/3,one/3,one/3]])
    b=snp.array([one/3,one/3,one/3])
    description=r"2nd-order, L-stable DIRK method of Bank et al."
    RK['TR-BDF2'] = RungeKuttaMethod(A, b, name='TR-BDF2',
                        description=description, shortname='TR-BDF2')
    return RK['TR-BDF2']

def loadRK_new(which='All'):
    from sympy import sqrt, Rational

    RK={}

    half = Rational(1,2)
    one  = Rational(1,1)
    zero = Rational(0,1)
    
    #####
    
    gamma =  2-sqrt(2)
    d = gamma/2
    w = sqrt(2)/4
    
    A=snp.array([[0,0,0],[d,d,0],[w,w,d]])
    b=snp.array([w,w,d])
    bhat=np.array([(1-w)/3,(3*w+1)/3,d/3])
    
    description=r"2nd-order, L-stable DIRK method of Bank et al. (In formulation of Hosea and Shampine)"
    RK['TR-BDF2'] = ImplicitRungeKuttaPair(A, b,bhat, name='TR-BDF2',
                        description=description, shortname='TR-BDF2')
    
    A=snp.array([[0,0,0],[one/4,one/4,0],[one/4,half,one/4]])
    b=snp.array([one/4,half,one/4])
    bhat=snp.array([one/6,(2*one)/3,one/6])
    
    description=r"2nd-order, A-stable? DIRK method of Hosea and Shampine)"
    RK['TRX'] = ImplicitRungeKuttaPair(A, b,bhat, name='TRX',
                        description=description, shortname='TRX')
    
    #####
    if which=='All':
        return RK
    else:
        return RK[which]

trbdf = loadRK_new('TR-BDF2').__num__()

In [ ]:
ex3 = rk.extrap(3,'implicit euler').__num__()
ex3.bhat=np.array([-1,1,1,0,0,0])

Note: There are two versions of this problem. 
In the simple one reaction 11 is deactuvated. In the second one the reaction is activated. 
Both need different inital conditions

In [ ]:
@jit(float64(float64),nopython=True)
def sigma(t):
    t_start = 12 #To make it start at noon
    T = ((t/3600)+t_start)%24
    Tr = 4.5
    Ts = 19.5

    if Tr <= T <= Ts:
        return 0.5+0.5*np.cos(np.pi*np.abs((2*T-Tr-Ts)/(Ts-Tr))*((2*T-Tr-Ts)/(Ts-Tr)))
    else:
        return 0


@jit(float64[:](float64,float64[:]),nopython=True)
def f_stratospheric_reac_alt(t,u):
    sig = sigma(t)
    print(sig)
    M = 8.120e16
    
    k1 = 2.643e-10*sig**3
    r1 = u[3]*k1

    k2 = 8.018e-17
    r2 = u[1]*u[3]*k2

    k3 = 6.12e-04*sig
    r3 = u[2]*k3

    k4 = 1.576e-15
    r4 = u[1]*u[2]*k4

    k5 = 1.070e-03*sig**2
    r5 = u[2]*k5

    k6 = 7.110e-11
    r6 = u[0]*M*k6

    k7 = 1.200e-10
    r7 = u[0]*u[2]*k7

    k8 = 6.062e-15
    r8 = u[4]*u[2]*k8

    k9 = 1.069e-11
    r9 = u[5]*u[1]*k9

    k10 = 1.289e-2*sig 
    r10 = u[5]*k10
    
    k11 = 1.0e-8
    r11 = u[4]*u[1]*k11
    #r11=0
    
    #u= [O1D,O,O3,O2,NO,NO2]
    #    0   1 2  3  4  5

    du = np.zeros(6)
    du[0] = -r6-r7           +r5                           #O1d
    du[1] = -r2-r4-r9        +2*r1+r3+r6+r10        -r11   #O 
    du[2] = -r3-r4-r5-r7-r8  +r2                           #O3
    du[3] = -r1-r2+r3        +2*r4+r5+2*r7+r8+r9           #O2
    du[4] = -r8              +r9+r10                -r11   #NO
    du[5] = -r9-r10          +r8                    +r11   #NO2
    
    return du


name = ['O1D','O','O3','O2','NO','NO2']
name_tex = ['$O^{1D}$','$O$','$O_3$','$O_2$','$NO$','$NO_2$']


@jit(float64[:](float64,float64[:]),nopython=True)
def f_stratospheric_reac(t,u):
    sig = sigma(t)
    
    [O1D,O,O3,O2,NO,NO2] =u
    
    k1 = 2.643e-10*sig**3
    r1 = O2*k1

    k2 = 8.018e-17
    r2 = O*O2*k2

    k3 = 6.120e-4*sig
    r3 = O3*k3

    k4 = 1.567e-15
    r4 = O3*O*k4

    k5 = 1.070e-3*sig**2
    r5 = O3*k5

    M  = 8.120e16
    k6= 7.110e-11
    r6= O1D*M*k6

    k7 = 1.200e-10
    r7 = O1D*O3*k7

    k8 = 6.062e-15
    r8 = O3*NO*k8

    k9 = 1.069e-11
    r9 = NO2*O*k9

    k10=1.289e-2*sig
    r10=NO2*k10

    k11=1.0e-8
    r11=NO*O*k11
    r11 =0

    
    dO1D =                      +r5 -r6 -  r7         
    dO   =  +2*r1 -r2 +r3 -  r4     +r6           -r9 +r10 -r11
    dO3  =        +r2 -r3 -  r4 -r5     -  r7 -r8    
    dO2  =  -  r1 -r2 +r3 +2*r4 +r5     +2*r7 +r8 +r9
    dNO  =                                    -r8 +r9 +r10 -r11
    dNO2 =                                    +r8 -r9 -r10 +r11
    
    return np.array([dO1D,dO,dO3,dO2,dNO,dNO2])


In [ ]:
t = np.linspace(0,3600*24,10000)
sig = np.zeros_like(t)
for i in range(len(t)):
    sig[i] = sigma(t[i])
plt.plot(t,sig)

In [ ]:
u0_simple = np.array([9.906e+1,6.624e8,5.326e11,1.697e16,8.725e8,2.240e8])
u0_complex = np.array([9.906e+1,6.624e08,5.326e11,1.697e16,4.000e6,1.093e9])
#                 u0 = [9.906e01,6.624e08,5.326e11,1.697e16,4.000e06,1.093e09]

u0 = u0_simple
du = f_stratospheric_reac(0,u0)
#Test if conservative
sum_o = np.array([1,1,3,2,1,2])
sum_n = np.array([0,0,0,0,1,1])
print(du@sum_o)
print(du@sum_n)

In [ ]:
# Step size control


beta1 = (0.7/5)
beta2 = (0.3/5)
#compute a totla error out of error and change and use this as inmut for a single PI controller
def dt_logic_PI(stepsize_control,dt_old,dt_adp,error,change,success,tol_met):
    print('error:',error)

    facmax = 1.2
    facmin = 0.1
    fac = 10
    w_change = 1
    #compute total error
    if change == None:
        change = 10  #maybe set her some other value
    error_tot = error + w_change*change
    
    dt_old = dt_logic_PI.dt_old 

    #Control
    Tol = stepsize_control.a_tol
    dt = dt_old *(Tol/error_tot)**beta1*(dt_logic_PI.error_tot_old[-1]/error_tot)**beta2
    
    #Update storage vaiables
    dt_logic_PI.error_tot_old[:-1] = dt_logic_PI.error_tot_old[1:] 
    dt_logic_PI.error_tot_old[-1] = error_tot
    
    
    print('Error/Tol:',error_tot/Tol)
    print('dt_suggest:',dt)
    #Correct for negative values
    if success:
        fac_pos = facmax
    else:
        fac_pos = 0.5
    
    dt = min(facmax*dt_old,fac_pos*dt_old,max(facmin*dt_old,dt))
    dt_logic_PI.dt_old=dt
    return max(dt,stepsize_control.dt_min)


dt_logic_PI.error_tot_old = np.zeros(3)
dt_logic_PI.dt_old=300

In [ ]:
dt_logic_PI.error_tot_old = np.zeros(3)
dt_logic_PI.dt_old=300

solver = Solver(rkm = ex3,
               dt = 300,
               t_final = 3600*72,
               b_fixed=False,
               tol_neg=1,
               tol_change = np.inf,
               p = [3,2,1],
               theta = [1],
               solver = cp.ECOS,
               convex=False,
               solver_eqs=solver_nonlinear_arg,
               LP_opts = {'reduce':True},
               fail_on_requect=True)

problem = Problem(f=f_stratospheric_reac,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

control = StepsizeControl(dt_min = 0.1,dt_max = np.infty,a_tol = 1e3,r_tol=1,f = dt_logic_PI
                          ,tol_reqect = 1e7)


status,t,u,b = RK_integrate(solver=solver,problem=problem,stepsize_control=control,verbose=True,dumpK=False)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
for i in range(6):
    plt.figure()
    plt.plot(t,u[i,:],'-x',label = name_tex[i])
    plt.legend()



In [ ]:
print(np.sum(u>0))
print(np.min(u))

In [ ]:
plt.plot(b.T)

In [ ]:
print(trbdf)

In [ ]:
trbdf.plot_stability_region()

In [ ]:
rk.loadRKM()

In [ ]:
#I am interested inthe correlation between these points
change = np.array(status['change'])
error = np.array(status['error'])

old_min = np.array(status['old_min'])

tol_met = np.array(status['sc'])
neg = np.array(status['b'])

plt.plot(error,change,'x',label = 'regular step')
plt.xlabel('Error')
plt.ylabel('Change')

#plot step reqects and infeasible
plt.plot(error[tol_met!='m'],change[tol_met!='m'],'rx',label = 'tol not met')
plt.plot(error[neg=='r'],change[neg=='r'],'ro',label = 'infeasible')
#at the moment there are no because we do not limit the change

plt.xlim([-0.1,2])
plt.ylim([-0.1,2])
plt.grid()


plt.figure()
plt.plot(old_min,change,'x')
plt.xlabel('Old Min')
plt.ylabel('Change')

plt.ylim([-0.1,2])
plt.xlim([-0.02,0.001])
plt.grid()